In [ ]:
! pip install -q pyspark==3.1.2 spark-nlp

     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 142 kB 62.7 MB/s 
     |████████████████████████████████| 198 kB 20.1 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
tamil=pd.read_excel('tamil.xlsx')
del tamil['படம்']
tamil_array=tamil[' தலைப்பு'].to_numpy()
print(tamil_array[0])

 இளஞ்சிவப்பு நிற உடையில் ஒரு குழந்தை நுழையும் வழியில் படிக்கட்டுகளில் ஏறுகிறது.


In [ ]:
tamil_sentences =' '.join(tamil_array)
print(tamil_sentences)

 இளஞ்சிவப்பு நிற உடையில் ஒரு குழந்தை நுழையும் வழியில் படிக்கட்டுகளில் ஏறுகிறது.  ஒரு பெண் ஒரு மர கட்டிடத்திற்குள் செல்கிறாள்.  ஒரு சிறிய பெண் ஒரு மர விளையாட்டு இல்லத்தில் ஏறுகிறாள்.  ஒரு சிறுமி தன் விளையாட்டு விடுதிக்கு படிக்கட்டுகளில் ஏறுகிறாள்.  இளஞ்சிவப்பு நிற உடையில் ஒரு சிறுமி மர அறைக்குள் செல்கிறாள்.  ஒரு கருப்பு நாய் மற்றும் ஒரு புள்ளி நாய் சண்டையிடுகின்றன  ஒரு கருப்பு நாயும், மூன்று நிற நாயும் சாலையில் ஒன்றோடு ஒன்று விளையாடிக் கொண்டிருந்தன.  ஒரு கருப்பு நாயும் பழுப்பு நிறப் புள்ளிகளுடன் ஒரு வெள்ளை நாயும் தெருவில் ஒருவரையொருவர் பார்த்துக் கொண்டிருக்கின்றன.  வெவ்வேறு இனத்தைச் சேர்ந்த இரண்டு நாய்கள் சாலையில் ஒன்றையொன்று பார்த்துக்கொண்டிருக்கின்றன.  நடைபாதையில் இரண்டு நாய்கள் ஒன்றையொன்று நோக்கி நகர்கின்றன. வர்ணம் பூசப்பட்ட ஒரு சிறுமி ஒரு கிண்ணத்தில் கைகளை வைத்து வர்ணம் பூசப்பட்ட வானவில்லின் முன் அமர்ந்திருக்கிறாள்.  ஒரு பெரிய வர்ணம் பூசப்பட்ட வானவில்லின் முன் ஒரு சிறுமி அமர்ந்திருக்கிறாள்.  புல்வெளியில் ஒரு சிறிய பெண் வானவில்லுடன் வெள்ளை கேன்வாஸின் முன் கைரேகையுடன் விளையாடுகிறாள். 

In [ ]:
import string     
print(string.punctuation)   

def punctuation_remove(text_data): 
    # Appending non punctuated words
    punctuation ="".join([t for t in text_data if t not in string.punctuation])  
    return punctuation
punctuation_removed_tamil_sentences=punctuation_remove(tamil_sentences)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
punctuation_removed_tamil_sentences

' இளஞ்சிவப்பு நிற உடையில் ஒரு குழந்தை நுழையும் வழியில் படிக்கட்டுகளில் ஏறுகிறது  ஒரு பெண் ஒரு மர கட்டிடத்திற்குள் செல்கிறாள்  ஒரு சிறிய பெண் ஒரு மர விளையாட்டு இல்லத்தில் ஏறுகிறாள்  ஒரு சிறுமி தன் விளையாட்டு விடுதிக்கு படிக்கட்டுகளில் ஏறுகிறாள்  இளஞ்சிவப்பு நிற உடையில் ஒரு சிறுமி மர அறைக்குள் செல்கிறாள்  ஒரு கருப்பு நாய் மற்றும் ஒரு புள்ளி நாய் சண்டையிடுகின்றன  ஒரு கருப்பு நாயும் மூன்று நிற நாயும் சாலையில் ஒன்றோடு ஒன்று விளையாடிக் கொண்டிருந்தன  ஒரு கருப்பு நாயும் பழுப்பு நிறப் புள்ளிகளுடன் ஒரு வெள்ளை நாயும் தெருவில் ஒருவரையொருவர் பார்த்துக் கொண்டிருக்கின்றன  வெவ்வேறு இனத்தைச் சேர்ந்த இரண்டு நாய்கள் சாலையில் ஒன்றையொன்று பார்த்துக்கொண்டிருக்கின்றன  நடைபாதையில் இரண்டு நாய்கள் ஒன்றையொன்று நோக்கி நகர்கின்றன வர்ணம் பூசப்பட்ட ஒரு சிறுமி ஒரு கிண்ணத்தில் கைகளை வைத்து வர்ணம் பூசப்பட்ட வானவில்லின் முன் அமர்ந்திருக்கிறாள்  ஒரு பெரிய வர்ணம் பூசப்பட்ட வானவில்லின் முன் ஒரு சிறுமி அமர்ந்திருக்கிறாள்  புல்வெளியில் ஒரு சிறிய பெண் வானவில்லுடன் வெள்ளை கேன்வாஸின் முன் கைரேகையுடன் விளையாடுகிறாள்  வானவில் ஓவி

In [ ]:
def remove_empty_strings(x):
    d =[]
    for i in x:
        if i !='':
            d.append(i)
    return d
tokenized_tamil_words = punctuation_removed_tamil_sentences.split(" ")
tokenized_tamil_words = remove_empty_strings(tokenized_tamil_words)
print(tokenized_tamil_words)

['இளஞ்சிவப்பு', 'நிற', 'உடையில்', 'ஒரு', 'குழந்தை', 'நுழையும்', 'வழியில்', 'படிக்கட்டுகளில்', 'ஏறுகிறது', 'ஒரு', 'பெண்', 'ஒரு', 'மர', 'கட்டிடத்திற்குள்', 'செல்கிறாள்', 'ஒரு', 'சிறிய', 'பெண்', 'ஒரு', 'மர', 'விளையாட்டு', 'இல்லத்தில்', 'ஏறுகிறாள்', 'ஒரு', 'சிறுமி', 'தன்', 'விளையாட்டு', 'விடுதிக்கு', 'படிக்கட்டுகளில்', 'ஏறுகிறாள்', 'இளஞ்சிவப்பு', 'நிற', 'உடையில்', 'ஒரு', 'சிறுமி', 'மர', 'அறைக்குள்', 'செல்கிறாள்', 'ஒரு', 'கருப்பு', 'நாய்', 'மற்றும்', 'ஒரு', 'புள்ளி', 'நாய்', 'சண்டையிடுகின்றன', 'ஒரு', 'கருப்பு', 'நாயும்', 'மூன்று', 'நிற', 'நாயும்', 'சாலையில்', 'ஒன்றோடு', 'ஒன்று', 'விளையாடிக்', 'கொண்டிருந்தன', 'ஒரு', 'கருப்பு', 'நாயும்', 'பழுப்பு', 'நிறப்', 'புள்ளிகளுடன்', 'ஒரு', 'வெள்ளை', 'நாயும்', 'தெருவில்', 'ஒருவரையொருவர்', 'பார்த்துக்', 'கொண்டிருக்கின்றன', 'வெவ்வேறு', 'இனத்தைச்', 'சேர்ந்த', 'இரண்டு', 'நாய்கள்', 'சாலையில்', 'ஒன்றையொன்று', 'பார்த்துக்கொண்டிருக்கின்றன', 'நடைபாதையில்', 'இரண்டு', 'நாய்கள்', 'ஒன்றையொன்று', 'நோக்கி', 'நகர்கின்றன', 'வர்ணம்', 'பூசப்பட்ட', 'ஒரு', 'சிறுமி', 'ஒரு', 

In [ ]:
f = open('TamilStopWords.txt',encoding="utf-8")
stop_words = f.readlines()
stop_words =[s.split("\n")[0] for s in stop_words]
print(stop_words)


['ஒரு', 'என்று', 'மற்றும்', 'இந்த', 'இது', 'என்ற', 'கொண்டு', 'என்பது', 'பல', 'ஆகும்', 'அல்லது', 'அவர்', 'நான்', 'உள்ள', 'அந்த', 'இவர்', 'என', 'முதல்', 'என்ன', 'இருந்து', 'சில', 'என்', 'போன்ற', 'வேண்டும்', 'வந்து', 'இதன்', 'அது', 'அவன்', 'தான்', 'பலரும்', 'என்னும்', 'மேலும்', 'பின்னர்', 'கொண்ட', 'இருக்கும்', 'தனது', 'உள்ளது', 'போது', 'என்றும்', 'அதன்', 'தன்', 'பிறகு', 'அவர்கள்', 'வரை', 'அவள்', 'நீ', 'ஆகிய', 'இருந்தது', 'உள்ளன', 'வந்த', 'இருந்த', 'மிகவும்', 'இங்கு', 'மீது', 'ஓர்', 'இவை', 'இந்தக்', 'பற்றி', 'வரும்', 'வேறு', 'இரு', 'இதில்', 'போல்', 'இப்போது', 'அவரது', 'மட்டும்', 'இந்தப்', 'எனும்', 'மேல்', 'பின்', 'சேர்ந்த', 'ஆகியோர்', 'எனக்கு', 'இன்னும்', 'அந்தப்', 'அன்று', 'ஒரே', 'மிக', 'அங்கு', 'பல்வேறு', 'விட்டு', 'பெரும்', 'அதை', 'பற்றிய', 'உன்', 'அதிக', 'அந்தக்', 'பேர்', 'இதனால்', 'அவை', 'அதே', 'ஏன்', 'முறை', 'யார்', 'என்பதை', 'எல்லாம்', 'மட்டுமே', 'இங்கே', 'அங்கே', 'இடம்', 'இடத்தில்', 'அதில்', 'நாம்', 'அதற்கு', 'எனவே', 'பிற', 'சிறு', 'மற்ற', 'விட', 'எந்த', 'எனவும்', 'எனப்படும்', 'எனி

In [ ]:
def stopwords_remove(text_data,stopwords):
    # Appending words which are not stop words  
    removed= [s for s in text_data if s not in stopwords]  
    return removed
stopwords_removed = stopwords_remove(tokenized_tamil_words,stop_words)

In [ ]:
print(stopwords_removed)

['இளஞ்சிவப்பு', 'நிற', 'உடையில்', 'குழந்தை', 'நுழையும்', 'வழியில்', 'படிக்கட்டுகளில்', 'ஏறுகிறது', 'பெண்', 'மர', 'கட்டிடத்திற்குள்', 'செல்கிறாள்', 'சிறிய', 'பெண்', 'மர', 'விளையாட்டு', 'இல்லத்தில்', 'ஏறுகிறாள்', 'சிறுமி', 'விளையாட்டு', 'விடுதிக்கு', 'படிக்கட்டுகளில்', 'ஏறுகிறாள்', 'இளஞ்சிவப்பு', 'நிற', 'உடையில்', 'சிறுமி', 'மர', 'அறைக்குள்', 'செல்கிறாள்', 'கருப்பு', 'நாய்', 'புள்ளி', 'நாய்', 'சண்டையிடுகின்றன', 'கருப்பு', 'நாயும்', 'மூன்று', 'நிற', 'நாயும்', 'சாலையில்', 'ஒன்றோடு', 'ஒன்று', 'விளையாடிக்', 'கொண்டிருந்தன', 'கருப்பு', 'நாயும்', 'பழுப்பு', 'நிறப்', 'புள்ளிகளுடன்', 'வெள்ளை', 'நாயும்', 'தெருவில்', 'ஒருவரையொருவர்', 'பார்த்துக்', 'கொண்டிருக்கின்றன', 'வெவ்வேறு', 'இனத்தைச்', 'இரண்டு', 'நாய்கள்', 'சாலையில்', 'ஒன்றையொன்று', 'பார்த்துக்கொண்டிருக்கின்றன', 'நடைபாதையில்', 'இரண்டு', 'நாய்கள்', 'ஒன்றையொன்று', 'நோக்கி', 'நகர்கின்றன', 'வர்ணம்', 'பூசப்பட்ட', 'சிறுமி', 'கிண்ணத்தில்', 'கைகளை', 'வைத்து', 'வர்ணம்', 'பூசப்பட்ட', 'வானவில்லின்', 'முன்', 'அமர்ந்திருக்கிறாள்', 'பெரிய', 'வர்ணம்', 'பூசப்

In [ ]:
#stopwords_removed_sentance = stopwords_remove(tamil_sentences.split(' '),stop_words)
stopwords_removed_sentance =" ".join(stopwords_removed)

In [ ]:
spark = sparknlp.start()
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"]) \
    .setOutputCol("token")

lemmatizer = LemmatizerModel.pretrained("lemma", "ta") \
        .setInputCols(["token"]) \
        .setOutputCol("lemma")

pipeline = Pipeline(stages=[document_assembler, tokenizer, lemmatizer])

example = spark.createDataFrame([[stopwords_removed_sentance]], ["text"])

results = pipeline.fit(example).transform(example)

lemma download started this may take some time.
Approximate size to download 47.9 KB
[OK!]


In [ ]:
rows =results.select('lemma').collect()
lemmatized =[]
for row in rows[0]['lemma']:
   lemmatized.append(row['result'])

In [ ]:
f = open('lemmatized_tamil.txt','w')
f.writelines(lemmatized)
f.close()

In [ ]:
!pip3 install stanza

     |████████████████████████████████| 432 kB 3.2 MB/s 
     |████████████████████████████████| 175 kB 46.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=4f8035bd7561c3677c7b43f5a57954fed8f2f13148c86e24c8d5751e98162a56
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import stanza


In [ ]:
stanza.download('ta')

2022-03-15 08:19:20 INFO: Downloading default packages for language: ta (Tamil)...


2022-03-15 08:19:30 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
ta_pos_tagger = stanza.Pipeline('ta', processors='tokenize,pos,lemma')

2022-03-15 08:19:32 WARNING: Language ta package default expects mwt, which has been added
2022-03-15 08:19:32 INFO: Loading these models for language: ta (Tamil):
| Processor | Package |
-----------------------
| tokenize  | ttb     |
| mwt       | ttb     |
| pos       | ttb     |
| lemma     | ttb     |

2022-03-15 08:19:32 INFO: Use device: cpu
2022-03-15 08:19:32 INFO: Loading: tokenize
2022-03-15 08:19:32 INFO: Loading: mwt
2022-03-15 08:19:32 INFO: Loading: pos
2022-03-15 08:19:33 INFO: Loading: lemma
2022-03-15 08:19:33 INFO: Done loading processors!


In [ ]:
word_list=[]
lemma_word_list=[]
pos_list=[]
for i in range(100):
  doc = ta_pos_tagger(tamil_array[i])
  for sentence in doc.sentences:
      for word in sentence.words:
          word_list.append(word.text)
          lemma_word_list.append(word.lemma)
          pos_list.append(word.pos)
  print(i,end='\r')


/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


In [ ]:
data={'Word':word_list,'Lemma':lemma_word_list,'POS Tagging':pos_list}
df=pd.DataFrame.from_dict(data)
df.head(5)


,Word,Lemma,POS Tagging
0,இளஞ்சிவப்பு,இளஞ்சிவப்பு,NOUN
1,நிற,நிறம்,NOUN
2,உடையில்,உடை,NOUN
3,ஒரு,ஒரு,ADJ
4,குழந்தை,குழந்தை,NOUN


In [ ]:
df.to_csv('tamil_lemma_pos.csv')